## IMDb Sentiment Analysis using LSTM PyTorch

## Import required modules

## Set Seed and enable GPU

In [2]:
torch.manual_seed(42) #its good practice to add a seed value

### Important point to note

To switch to CUDA(GPU), follow the below steps:
- Click on Runtime
- Click on `Change runtime type`
- Select Hardware Accelerator as `T4 GPU`

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

## Download the Dataset using datasets library by HuggingFace

In [4]:
import datasets


In [2]:
pip install datasets

In [3]:
from datasets import load_dataset
dataset = load_dataset("imdb")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
import spacy
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# 🔄 Load spaCy English tokenizer
nlp = spacy.load("en_core_web_sm")

# 🔧 Tokenization and cleanup
def spacy_tokenize(text):
    doc = nlp(text.lower())
    return ' '.join([token.text for token in doc if token.is_alpha and not token.is_stop])

# 📥 Load IMDb dataset (subsample for speed)
dataset = load_dataset("imdb")
train_data = dataset["train"].shuffle(seed=42).select(range(5000))
test_data = dataset["test"].shuffle(seed=42).select(range(1000))

# 🧹 Apply tokenization
train_texts = [spacy_tokenize(example["text"]) for example in train_data]
train_labels = train_data["label"]

test_texts = [spacy_tokenize(example["text"]) for example in test_data]
test_labels = test_data["label"]



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

### Note: Why do we need UNK and PAD?

Let's say we have a large corpus of text data. During tokenization we usually fit in all the train data. When we have a new text, if the model encounters a new word, it will assign it as <UNK>, which stands for unknown.

Let's take a few sample movie reivews:
- I loved this movie
- Amazing
- Impressive storyline
- Terrible experience not recommended to watch

If you look at the above statements, all have different word sizes. To ensure we pass the model with the same size, we pad all the sentences to be in the same size. We set the max length to be some value if the sequence has more than the threshold it truncates the padding. If it is less than the sequence, it pad and fill the sequence with zero.

## Prepare the dataset for the model

In [ ]:
def convert_into_tokens(example, vocab):
    ids = [vocab[token] for token in example['tokens']]
    return {'ids': ids}

In [ ]:
#this data will be used for training
train_data = train_data.map(convert_into_tokens, fn_kwargs={'vocab': vocab})
# this data will be used for evaluation
valid_data = valid_data.map(convert_into_tokens, fn_kwargs={'vocab': vocab})
# this the data that we use for generalization [New unseen data for testing]
test_data = test_token.map(convert_into_tokens, fn_kwargs={'vocab': vocab})

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

## Look now we have `ids` as the next data appended

#### Token is the individual representation of the word and the ids is the equivalent numerical value to that token

In [ ]:
train_data['ids'][:10]

[[12,
  311,
  5,
  174,
  706,
  7,
  14,
  242,
  18,
  74,
  38,
  308,
  15,
  2,
  23,
  3467,
  28,
  21,
  106,
  210,
  6,
  15,
  11,
  17,
  650,
  71,
  0,
  675,
  3,
  12,
  939,
  15,
  432,
  4,
  2,
  23,
  277,
  28,
  5,
  236,
  106,
  210,
  25,
  0,
  68,
  31,
  88,
  211,
  3,
  3,
  3,
  24,
  22,
  12,
  474,
  94,
  29,
  109,
  11,
  0,
  2,
  860,
  579,
  3,
  1199,
  1326,
  9,
  0,
  769,
  1104,
  6,
  577,
  13094,
  96,
  1767,
  4688,
  4,
  4784,
  6,
  1475,
  2901,
  30,
  128,
  2102,
  13,
  5,
  14473,
  1365,
  3,
  2,
  18,
  314,
  100,
  65,
  13,
  842,
  90,
  1326,
  10994,
  6635,
  25,
  4580,
  0,
  9,
  16,
  6,
  2537,
  13107,
  9,
  16,
  126,
  30,
  0,
  13,
  15,
  579,
  24,
  22,
  99,
  13,
  11443,
  41,
  7823,
  7983,
  13,
  1326,
  9,
  6542,
  7,
  660,
  3,
  11,
  10,
  0,
  8,
  73,
  41,
  0,
  4,
  0,
  6,
  0,
  6835,
  433,
  146,
  5,
  9245,
  6,
  2623,
  7686,
  3,
  432,
  2,
  18,
  169,
  9,
  27,
  3588,


In [ ]:
train_data = train_data.with_format(type='torch', columns=['ids', 'label', 'length'])
valid_data = valid_data.with_format(type='torch', columns=['ids', 'label', 'length'])
test_data = test_data.with_format(type='torch', columns=['ids', 'label', 'length'])

## Model building- LSTM

In [ ]:
class LSTMmodel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,dropout_rate, pad_index):
        super().__init__()
        # layer 1- Pass the ids to the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        # layer 2- LSTM [If n_layers = 2, then layer 3 is also LSTM]
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,dropout=dropout_rate, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate) # to avoid overfitting

    def forward(self, ids, batch_size):
        # token to embeddings
        embedded = self.dropout(self.embedding(ids))
        embdedded = nn.utils.rnn.pack_padded_sequence(embedded, batch_size, batch_first=True,enforce_sorted=False)
        # embedding sequence (batch_size,seq_length,emd_dim) to LSTM

        outputs, (hidden, cell) = self.lstm(embdedded)

        output, output_length = nn.utils.rnn.pad_packed_sequence(outputs)
        hidden = self.dropout(hidden[-1])

        prediction = self.fc(hidden)
        return prediction

In [ ]:
vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 64
output_dim = len(train_data.unique('label')) # either 0 or 1 = 2(length)
n_layers = 2
dropout_rate = 0.5

model = LSTMmodel(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout_rate,vocab['<PAD>'])
model = model.to(device) #switch our modeling training in GPU

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad) # total parameters

1945474

In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.orthogonal_(param)

In [ ]:
model.apply(initialize_weights)

LSTMmodel(
  (embedding): Embedding(14550, 128, padding_idx=1)
  (lstm): LSTM(128, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

## Using Pre-trained embeddings

Few of the most used Pre-trained embedding are:
- GloVe
- Word2Vec
- FastText

In [ ]:
vectors = torchtext.vocab.GloVe() # extra pre-trained embebedding

.vector_cache/glove.840B.300d.zip: 2.18GB [06:51, 5.29MB/s]                            
100%|█████████▉| 2196016/2196017 [05:31<00:00, 6631.53it/s]


In [ ]:
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())

In [ ]:
model.embedding.weight.data = pretrained_embedding

## Compile Model

- Three important parameter that influence the model are:
  - Optimizer- algorithm for gradient descent [Adam, SGD, RMSProp]
  - Loss function- Binary cross entropy loss or CrossEntropy loss
  - Evaluation performance metrics [Accuracy, Precision, Recall]

In [ ]:
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss().to(device)

def metrics(prediction, actual):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(actual).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

In [ ]:
def collate(batch, pad_index):
    batch_ids = [i['ids'] for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    batch_length = [i['length'] for i in batch]
    batch_length = torch.stack(batch_length)
    batch_label = [i['label'] for i in batch]
    batch_label = torch.stack(batch_label)
    batch = {'ids': batch_ids,
             'length': batch_length,
             'label': batch_label}
    return batch

## Fit the data to the model

In [ ]:
batch_size = 64
collate = functools.partial(collate, pad_index= vocab['<PAD>'])

train_dataloader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               collate_fn=collate,
                                               shuffle=True)

valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, collate_fn=collate)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, collate_fn=collate)

## Train the model

In [ ]:
def train(dataloader, model, loss_function, optimizer, device):
    model.train()

    epoch_losses = []
    epoch_accs = []

    for batch in tqdm.tqdm(dataloader, desc='training...', file=sys.stdout):
        ids = batch['ids'].to(device)

        #batch length - dataloader
        length = batch['length']
        label = batch['label'].to(device)

        # y_hat = prediction from the model
        prediction = model(ids, length)
        # loss function - > Actual value, predicted value
        # actual value - label
        # predicted value is prediction
        loss = loss_function(prediction, label) #loss

        accuracy = metrics(prediction, label) #
        optimizer.zero_grad() #adam -> gradient descent

        loss.backward()
        optimizer.step() #we will update the weights with learning rate

        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())

    return epoch_losses, epoch_accs

## Evaluation

In [ ]:
def evaluate(dataloader, model, loss_function, device):

    model.eval()
    epoch_losses = []
    epoch_accs = []

    with torch.no_grad(): # no optimization -> no update in weightds
        for batch in tqdm.tqdm(dataloader, desc='evaluating...', file=sys.stdout):
            ids = batch['ids'].to(device)
            length = batch['length']
            label = batch['label'].to(device)
            prediction = model(ids, length)
            loss = loss_function(prediction, label)
            accuracy = metrics(prediction, label)
            epoch_losses.append(loss.item())
            epoch_accs.append(accuracy.item())

    return epoch_losses, epoch_accs

In [ ]:
n_epochs = 5
best_valid_loss = float('inf')

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(n_epochs):

    train_loss, train_acc = train(train_dataloader, model, loss_function, optimizer, device)
    valid_loss, valid_acc = evaluate(valid_dataloader, model, loss_function, device)

    train_losses.extend(train_loss)
    train_accs.extend(train_acc)
    valid_losses.extend(valid_loss)
    valid_accs.extend(valid_acc)

    epoch_train_loss = np.mean(train_loss)
    epoch_train_acc = np.mean(train_acc)
    epoch_valid_loss = np.mean(valid_loss)
    epoch_valid_acc = np.mean(valid_acc)

    if epoch_valid_loss < best_valid_loss:
        best_valid_loss = epoch_valid_loss
        torch.save(model.state_dict(), 'lstm.pt')

    print(f'Epoch: {epoch+1}/{n_epochs}')
    print(f'loss: {epoch_train_loss:.4f}, accuracy: {epoch_train_acc:.4f}')
    print(f'valid_loss: {epoch_valid_loss:.4f}, valid_accuracy: {epoch_valid_acc:.4f}')
    print("--"*25)

evaluating...: 100%|██████████| 79/79 [00:00<00:00, 86.78it/s]
Epoch: 1/5
loss: 0.6610, accuracy: 0.6089
valid_loss: 0.6248, valid_accuracy: 0.6715
--------------------------------------------------
evaluating...: 100%|██████████| 79/79 [00:00<00:00, 87.75it/s]
Epoch: 2/5
loss: 0.6485, accuracy: 0.6321
valid_loss: 0.6228, valid_accuracy: 0.6622
--------------------------------------------------
evaluating...: 100%|██████████| 79/79 [00:01<00:00, 54.08it/s]
Epoch: 3/5
loss: 0.6367, accuracy: 0.6457
valid_loss: 0.6678, valid_accuracy: 0.6503
--------------------------------------------------
evaluating...: 100%|██████████| 79/79 [00:01<00:00, 60.53it/s]
Epoch: 4/5
loss: 0.6598, accuracy: 0.6246
valid_loss: 0.6359, valid_accuracy: 0.6493
--------------------------------------------------
evaluating...: 100%|██████████| 79/79 [00:00<00:00, 89.23it/s]
Epoch: 5/5
loss: 0.6447, accuracy: 0.6339
valid_loss: 0.6369, valid_accuracy: 0.6596
--------------------------------------------------


## Evaluation of the model

In [ ]:
model.load_state_dict(torch.load('lstm.pt')) #save the models

test_loss, test_acc = evaluate(test_dataloader, model, loss_function, device)

epoch_test_loss = np.max(test_loss)
epoch_test_acc = np.max(test_acc)

print("Loss",epoch_test_loss)
print("Acc",epoch_test_acc)

evaluating...: 100%|██████████| 391/391 [00:05<00:00, 76.77it/s]
Loss 0.811225414276123
Acc 0.921875


## Predictions on User Input

In [ ]:
def make_prediction(text, model, tokenizer, vocab):
    #find the token for the user input
    tokens = tokenizer(text)
    #convert token into numerical number (unique id)
    ids = [vocab[t] for t in tokens]

    #find the length and convert the ids into tensor to feed in LSTM model
    length = torch.LongTensor([len(ids)])
    tensor = torch.LongTensor(ids).unsqueeze(dim=0).to(device)

    #make prediction
    prediction = model(tensor, length).squeeze(dim=0)
    probability = torch.softmax(prediction, dim=-1) #check for the score - probability (softmax)

    predicted_class = prediction.argmax(dim=-1)
    predicted_probability = probability[predicted_class]
    return predicted_class, predicted_probability

In [ ]:
def display(label,score):
    if label==0:
        print(f"Negative-Score:{score}")
    else:
        print(f"Positive-Score:{score}")

In [ ]:
text = "Amazing movie, loved it"
label,score = make_prediction(text, model, tokenizer, vocab)
display(label,score)

Positive-Score:0.547923743724823
